# Qiskit Patterns

In this how-to we will learn about Qiskit patterns and quantum approximate optimization. Qiskit Patterns are a workflow to execute a task on a Quantum computer. They comprise four steps

1. Map the classical inputs to a quantum problem
2. Optimize problem for quantum execution
3. Execute using Qiskit Runtime primitives
4. Post-process, return result in classical format

We will apply the patterns to the context of **combinatorial optimization** and show how to solve a problem using the **Quantum Approximate Optimization Algorithm (QAOA)**, a hybrid (quantum-classical) iterative method. 

- In Step 1. we will take a combinatorial optimization problem and formulate it in terms of finding the ground state of an Ising Hamiltonian. This reformulated problem can be understood by a quantum computer.
- In Step 2. we will prepare the quantum circuits to execute on the quantum computer.
- In Step 3. we will iterativelly call the `Sampler` primitive in Qiskit to draw samples from the quantum circuits that Step 2. prepared., and use those samples in the loss function of our algorithmic routine
- Finally, under Step 4. we convert the samples from Step 3. into the solution of our combinatorial optimization problem. 

## 1. Map the classical inputs to a quantum problem

We are interested in solving a classical combinatorial optimization problem which has the form

\begin{align}
\min_{x\in \{0, 1\}^n}f(x)
\end{align}

Here, the vector $x$ are the $n$ decision variables. As you can see, there is nothing relating to quantum computing here yet. We therefore need to reformulate this problem into something that a quantum computer can understand. To be more concrete, we will consider a Quadratic Unconstrained Binary Optimization problem with the form

\begin{align}
\min_{x\in \{0, 1\}^n}x^T Q x,
\end{align}

where $Q$ is a $n\times n$ matrix of real numbers. To start, we will convert the binary variables $x_i$ to variables $z_i\in\{-1, 1\}$ by doing

\begin{align}
x_i = \frac{1-z_i}{2}.
\end{align}

Here, for example, we see that if $x_i$ is $0$ then $z_i$ is $1$. When we substitute the $x_i$'s for the $z_i$'s in the QUBO above, we obtain the equivalent formulations for our optimization task

\begin{align}
\min_{x\in\{0,1\}^n} x^TQx\Longleftrightarrow \min_{z\in\{-1,1\}^n}z^TQz + b^Tz
\end{align}

The details of the computation are shown in Appendix A below. Here, $b$ depends on $Q$. Note that to obtain $z^TQz + b^Tz$ we dropped an irrelevant factor of 1/4 and a constant offset of $n^2$ which do not play a role in the optimization. Now, to obtain a quantum formulation of the problem we promot the $z_i$ variables to a Pauli $Z$ matrix, i.e., a $2\times 2$ matrix of the form

\begin{align}
Z_i = \begin{pmatrix}1 & 0 \\ 0 & -1\end{pmatrix}.
\end{align}

When we substitute these matrices in the QUBO above we obtain the following Hamiltonian

\begin{align}
H_C=\sum_{ij}Q_{ij}Z_iZ_j + \sum_i b_iZ_i.
\end{align}

We refer to this Hamiltonian as the **cost function Hamiltonian**. It has the property that its gound state corresponds to the solution that **minimizes the cost function $f(x)$**.
Therefore, to solve our optimization problem we now need to prepare the ground state of $H_C$ (or a state with a high overlap with it) on the quantum computer. Then, sampling from this state will, with a high probability, yield the solution to $min f(x)$.

In [6]:
# load a lp file
lp_file = "data/maxcut.lp"
with open(lp_file, "r") as file:
    problem = file.read()
print(problem)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: Max-cut

Maximize
 obj: 3 x_0 + 3 x_1 + 3 x_2 + 3 x_3 + 3 x_4 + 3 x_5 + 3 x_6 + 3 x_7 + 3 x_8
      + 3 x_9 + [ - 4 x_0*x_3 - 4 x_0*x_5 - 4 x_0*x_7 - 4 x_1*x_2 - 4 x_1*x_4
      - 4 x_1*x_6 - 4 x_2*x_3 - 4 x_2*x_8 - 4 x_3*x_5 - 4 x_4*x_6 - 4 x_4*x_9
      - 4 x_5*x_8 - 4 x_6*x_9 - 4 x_7*x_8 - 4 x_7*x_9 ]/2
Subject To

Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
 0 <= x_4 <= 1
 0 <= x_5 <= 1
 0 <= x_6 <= 1
 0 <= x_7 <= 1
 0 <= x_8 <= 1
 0 <= x_9 <= 1

Binaries
 x_0 x_1 x_2 x_3 x_4 x_5 x_6 x_7 x_8 x_9
End



### TODO: 
- [x] load LP file (use a dummy graph) and print it.
- [ ] load Ising hamiltonian and print first few terms.

## 2. Optimize problem for quantum execution

(*In Step 2. we will prepare the quantum circuits to execute on the quantum computer.*)

The Hamiltonian obtained from step 1 contains the quantum definition of our problem. To run QAOA, we need to convert this definition into a parametric circuit form. This will be the reference state for our iterative quantum execution. Step 2 of the Qiskit Patterns focuses on tuning the circuit for optimal quantum execution through **transpilation**. The Qiskit library offers a series of **Transpilation Passes** that cater to a wide range of circuit transformations. We don't only want to get a circuit, but we want to make sure that the circuit is **optimized** for our purpose. In the context of QAOA, we care about matching the qubit configuration of the quantum chip to the graph defined by the cost function Hamiltonian. By manually tuning this configuration, we can ensure that the final layout will not involve unnecessary operations that would increase the noise level of our samples.

**TODO**: Take the Ising Hamiltonian from Step 1. We are loading a dummy Hamiltonian in the meantime




The QAOA ansatz (circuit) is composed of a cost operator and a mixer operator applied in an alternating fashion to an initial state. The cost operator corresponds to the **cost Hamiltonian** defined in step 1, and is expressed as a weighted sum of pauli terms.

In [ ]:
import json
import os
import networkx as nx
from qopt_best_practices.utils import build_max_cut_graph, build_max_cut_paulis

In [ ]:
# get dummy Hamiltonian
graph_file = "data/graph_2layers_0seed.json"

with open(graph_file, "r") as file:
            data = json.load(file)

dummy_graph = nx.from_edgelist(data["Original graph"])
dummy_hamiltonian = build_max_cut_paulis(dummy_graph)

# inject dummy Hamiltonian into workflow
graph = dummy_graph
hamiltonian = dummy_hamiltonian
print(hamiltonian)

### 2.1. Define SWAP strategy

- What is a swap strategy
- Why is it relevant
- How do we apply it -> transpiler pass

In [ ]:
from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import SwapStrategy

swap_strategy = SwapStrategy.from_line([i for i in range(num_qubits)])

### 2.2 Build QAOA circuit applying previously defined SWAP strategy

IDEA: modify `create_qaoa_swap_circuit` to accept pass manager as input instead of constructing pass manager under the hood

In [ ]:
from qopt_best_practices.swap_strategies import create_qaoa_swap_circuit

# we define the edge_coloring map so that RZZGates are positioned next to SWAP gates to exploit CX cancellations
edge_coloring = {(idx, idx + 1): (idx + 1) % 2 for idx in range(qaoa_hamiltonian.num_qubits)}

qaoa_circ = create_qaoa_swap_circuit(qaoa_hamiltonian, swap_strategy, edge_coloring, qaoa_layers=2)

qaoa_circ.draw("mpl")

### 2.3 Define layout from backend

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel='ibm_quantum')
backend = service.get_backend('ibm_sherbrooke')

In [ ]:
from qopt_best_practices.qubit_selection import BackendEvaluator

path_finder = BackendEvaluator(backend)

# the Backend Evaluator accepts custom subset definitions and metrics,
# but defaults to finding the line with the best fidelity
path, fidelity, num_subsets = path_finder.evaluate(num_qubits)

print("Best path: ", path)
print("Best path fidelity", fidelity)
print("Num. evaluated paths", num_subsets)

In [ ]:
from qiskit.transpiler import Layout

initial_layout = Layout.from_intlist(path, qaoa_circ.qregs[0])

### 2.4 Apply layout and other transpiler passes

In [ ]:
from qiskit.transpiler import CouplingMap, PassManager
from qiskit.transpiler.passes import (
    FullAncillaAllocation,
    EnlargeWithAncilla,
    ApplyLayout,
    SetLayout,
)

from qiskit import transpile

basis_gates = ["rz", "sx", "x", "cx"]

backend_cmap = CouplingMap(backend.configuration().coupling_map)

pass_manager_post = PassManager(
    [
        SetLayout(initial_layout),
        FullAncillaAllocation(backend_cmap),
        EnlargeWithAncilla(),
        ApplyLayout(),
    ]
)

# Map to initial_layout and finally enlarge with ancilla.
qaoa_circ = pass_manager_post.run(qaoa_circ)

# Now transpile to sx, rz, x, cx basis
qaoa_circ = transpile(qaoa_circ, basis_gates=basis_gates)
qaoa_circ.draw("mpl", idle_wires=False)

## 3. Execute using Qiskit Runtime primitives

### 3.1 Define Sampler primitive

In [ ]:
from qiskit_ibm_runtime import Sampler, Options

options = Options()
options.transpiler.skip_transpilation = True

sampler = Sampler(backend=backend, options=options)

### 3.2 Define cost function

In [ ]:
from qopt_best_practices.cost_function import evaluate_sparse_pauli


def cost_func_sampler(params, ansatz, hamiltonian, sampler):

    job = sampler.run(ansatz, params)
    sampler_result = job.result()
    sampled = sampler_result.quasi_dists[0]

    # a dictionary containing: {state: (measurement probability, value)}
    evaluated = {
        state: (probability, evaluate_sparse_pauli(state, hamiltonian))
        for state, probability in sampled.items()
    }

    result = sum(probability * value for probability, value in evaluated.values())

    return result

### 3.3 Define initial point

In [ ]:
import numpy as np

# TQA initialization parameters
dt = 0.75
p = 2  # 2 qaoa layers
grid = np.arange(1, p + 1) - 0.5
init_params = np.concatenate((1 - grid * dt / p, grid * dt / p))
print(init_params)

### 3.4 Run optimization

In [ ]:
from scipy.optimize import minimize

result = minimize(
    cost_func_sampler,
    init_params,
    args=(qaoa_circ, qaoa_hamiltonian, sampler),
    method="COBYLA",
)
print(result)


## 4. Post-process, return result in classical format

**TODO** Plot the distribution of results.

## Discussion and conclusion

## Appendix A: Reformulation in spin variables

Here, we rewrite the QUBO $x^TQx$ in terms of spin-variables $x_i=(1-z_i)/2$.
\begin{align}
x^TQx=\sum_{ij}Q_{ij}x_ix_j=\frac{1}{4}\sum_{ij}Q_{ij}(1-z_i)(1-z_j)=\frac{1}{4}\sum_{ij}Q_{ij}z_iz_j-\frac{1}{4}\sum_{ij}(Q_{ij}+Q_{ji})z_i + \frac{n^2}{4}.
\end{align}
If we write $b_i=-\sum_{j}(Q_{ij}+Q_{ji})$ and remove the prefactor and the constant $n^2$ term we arrive at the two equivalent formulations of the same optimization problem
\begin{align}
\max_{x\in\{0,1\}^n} x^TQx\Longleftrightarrow \max_{z\in\{-1,1\}^n}z^TQz + b^Tz
\end{align}

## Appendix B: Quantum notation

The $Z$ matrices are imbedded in the quantum computer's computational space, i.e., a Hilbert space of size $2^n\times 2^n$. Therefore, you should understand terms such as $Z_iZ_j$ as the tensor product $Z_i\otimes Z_j$ imbedded in the $2^n\times 2^n$ Hilbert space. For example, in a problem with five decision variables the term $Z_1Z_3$ is understood to mean $I\otimes Z_3\otimes I\otimes Z_1\otimes I$ where $I$ is the $2\times 2$ identity matrix.